## Imports

In [36]:
#importing necessary libraries
from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

## Importing Coverage Function (allows me to pick any branch I want)

In [135]:
#reading in library branch and population demographics, both with geometry data
#branchInfo = pd.read_csv('../library neighborhoods/completedCyclingNeighborhoodRadiiDF.csv')

#branchInfo = pd.read_csv('../library neighborhoods/drive1branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive3minus1branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive5minus3branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive7minus5branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive9minus7branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive11minus9branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive13minus11branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive15minus13branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive17minus15branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/drive19minus17branchRadii.csv')
branchInfo = pd.read_csv('../library neighborhoods/drive21minus19branchRadii.csv')

censusTracts = pd.read_csv('../data/clean/population_demographics.csv')
censusDemographics = pd.read_csv('../data/dani data/census_demos.csv')

In [136]:
#removing unecessary columns from branchInfo
branchInfo = branchInfo.drop(columns = ['Unnamed: 0'])
#removing unecessary columns from censusTracts
censusTracts = censusTracts[['geoid','geometry','qualifying name']]

#converting censusTracts to a geodataframe (we could not load it as one since it already had a geometry column so we are simply
#converting the geometry column to a proper gpd geometry column).
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )

branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])

branchInfo = gpd.GeoDataFrame(
    branchInfo.loc[:, [c for c in branchInfo.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(branchInfo["geometry"]),
    crs="epsg:4326",
    )

In [110]:
censusPops = censusDemographics[['geoid', 'total population']].copy(deep=True)
censusPops

,geoid,total population
0,17031823303,4059
1,17031823304,6316
2,17031740400,4738
3,17031740300,5017
4,17031822802,3750
...,...,...
864,17031320400,3058
865,17031320101,7952
866,17031081403,8984
867,17031081402,5613


In [111]:
#helper method to ensure that two polygons overlap before getting the intersection. Gets the intersection then calculates
#the percent of overlapping area for a census tract and saves it to a dictionary with the census tract geo-id as the key and
#the percent as the value.
def check(polygon1, df, geoname, dictionary,key):
   
    for i in range(len(df)):
        if polygon1.intersects(df.loc[i,geoname]): 
            overlapPolygon = (polygon1.intersection(df.loc[i,geoname]))
            poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
            overlapArea = poly_area*-1
            propOverlap = overlapArea / df.loc[i,'Area']
            #print(propOverlap)
            dictionary[df.loc[i, key]] = propOverlap
    return dictionary
   

In [103]:
#sets the crs of gpd types
def geoSetup(gdf):
    
    gdf = gdf.set_crs('EPSG:4326')
    

In [104]:
#cdf = gdf of geometry area which you want to get the coverage score
#idf = gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs = a string that is the name of the geometry column in the cdf
#igs = a string that is the name of the geometry column in the idf
#lat = a string that is the name of the latitude column in the idf
#lon = a string that is the name of the longitude column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary

def appendADS(cdf,idf,scoreString,dictString,cgs,igs,lat,lon,key):
    
    #api = MapboxAPI()
    cdf[scoreString] = 0.0
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas



    for i in range(len(idf)):
        dictionary = {}
        dictionary = check(idf.loc[i,igs],cdf,cgs, dictionary,key)
        branchInfo.loc[i,dictString]= [dictionary]
    
        for i in range(len(cdf)):
   
            if dictionary.get(cdf.loc[i, key]) != None:
                score = cdf.loc[i, scoreString]
                cdf.loc[i, scoreString] = score + dictionary.get(cdf.loc[i, key])
  
   

In [137]:
appendADS(censusTracts,branchInfo, 'score', 'list of dict','geometry','geometry','LATITUDE','LONGITUDE','geoid')

/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt

In [85]:
branchInfo.head(3)

,BRANCH,LOCATION,LATITUDE,LONGITUDE,geometry,list of dict
0,Albany Park,"(41.97557881655979, -87.71361314512697)",41.975579,-87.713613,"POLYGON ((-87.71861 41.96972, -87.71911 41.971...","[{17031140702: 0.006020828811234736, 170311407..."
1,Altgeld,"(41.65719847558056, -87.59883829075173)",41.657198,-87.598838,"POLYGON ((-87.61484 41.65813, -87.61594 41.658...","[{17031540101: 0.14722604798242142, 1703155020..."
2,Archer Heights,"(41.80110836194246, -87.72648385568911)",41.801108,-87.726484,"MULTIPOLYGON (((-87.73105 41.79311, -87.73120 ...","[{17031835200: 0.23873361304596588, 1703156030..."


## Average Travel Time Function

In [90]:

#Internet
def min5Radius(DemographicsVariables, branchWithCensus):
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    #Branch_pop_withInternet = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['Total']).sum() 
    #result = Branch_pop_withInternet/pop_in_branch
    return pop_in_branch   

#Income


In [91]:
def branch_demographics(branchWithCensus, nameOfBranch):
    branch_snapshot_df = pd.DataFrame({'Branch': [nameOfBranch]})
    list1 = ['5 Min']
    for i in list1:
        branch_snapshot_df[i] = min5Radius(i, branchWithCensus)
    
    return branch_snapshot_df

In [78]:
def incrementPopulations(branchDataset):
    popPerLib = pd.DataFrame(columns=['Branch', 'Given Min Pop'])

    for i in range(len(branchDataset)):
        
        #Locating only Altgeld's column 'list of dict'
        libraryDictList = branchDataset.loc[i,'list of dict']
        branchName = branchDataset.loc[i,'BRANCH']
        
        #Removed set of brackets, just a dictionary now
        finalDictionary = libraryDictList[0]

        #Seperating into two columns
        dictToList = pd.DataFrame(finalDictionary.items(), columns= ['geoid','Percent Overlap'])
        libraryAndCensus = censusPops.merge(dictToList, on='geoid')

        libraryAndCensus['counts'] = 0
        for j in range(len(libraryAndCensus)):
            libraryAndCensus.loc[j,'counts'] = libraryAndCensus.loc[j,'total population'] * libraryAndCensus.loc[j,'Percent Overlap']
        
        libPop = pd.DataFrame({'BRANCH': [branchName],'Given Min Pop': libraryAndCensus['counts'].sum()})
        popPerLib = pd.concat([popPerLib, libPop])
        
    popPerLib = popPerLib.drop(columns = ['Branch'])
    popPerLib = popPerLib.reset_index()
    popPerLib = popPerLib.drop(columns = ['index'])
    
    return popPerLib

In [138]:
oneRing = incrementPopulations(branchInfo)

In [139]:
c21Min19Population = oneRing.copy(deep = True)
c21Min19Population

,Given Min Pop,BRANCH
0,0,Albany Park
1,0,Altgeld
2,0,Archer Heights
3,0,Austin
4,0,Austin-Irving
...,...,...
76,0,West Pullman
77,7.124673,West Town
78,17.346121,"Whitney M. Young, Jr."
79,0,Woodson Regional Library


In [162]:
allRadiiPops = pd.DataFrame(columns=['Branch', 'Prop 0 to 1 Min','Prop 1 to 3 Min', 'Prop 3 to 5 Min', 'Prop 5 to 7 Min', 'Prop 7 to 9 Min', 'Prop 9 to 11 Min', 
                                     'Prop 11 to 13 Min', 'Prop 13 to 15 Min', 'Prop 15 to 17 Min', 'Prop 17 to 19 Min',
                                     'Prop 19 to 21 Min', 'Total Pop Around Library'])

In [163]:
allRadiiPops['Branch'] = c1Min0Population['BRANCH']
allRadiiPops['Prop 0 to 1 Min'] = c1Min0Population['Given Min Pop']
allRadiiPops['Prop 1 to 3 Min'] = c3Min1Population['Given Min Pop']
allRadiiPops['Prop 3 to 5 Min'] = c5Min3Population['Given Min Pop']
allRadiiPops['Prop 5 to 7 Min'] = c7Min5Population['Given Min Pop']
allRadiiPops['Prop 7 to 9 Min'] = c9Min7Population['Given Min Pop']
allRadiiPops['Prop 9 to 11 Min'] = c11Min9Population['Given Min Pop']
allRadiiPops['Prop 11 to 13 Min'] = c13Min11Population['Given Min Pop']
allRadiiPops['Prop 13 to 15 Min'] = c15Min13Population['Given Min Pop']
allRadiiPops['Prop 15 to 17 Min'] = c17Min15Population['Given Min Pop']
allRadiiPops['Prop 17 to 19 Min'] = c19Min17Population['Given Min Pop']
allRadiiPops['Prop 19 to 21 Min'] = c21Min19Population['Given Min Pop']

allRadiiPops['Total Pop Around Library'] = 0

In [164]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library
0,Albany Park,743.233490,11461.490870,16937.056665,1629.263526,0,0,0,0,0,0,0,0
1,Altgeld,171.224109,1333.984489,3361.805928,13984.505678,441.4195,19.440495,0.213771,0,0,0,0,0
2,Archer Heights,1768.226150,10075.564142,11282.105777,3485.545272,564.116169,0,0,0,0,0,0,0
3,Austin,896.808562,14412.841196,13947.921464,7484.997177,15399.165208,0,0,0,0,0,0,0
4,Austin-Irving,1419.246097,13355.359822,11761.164473,198.400952,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,768.427459,8982.612539,32331.153537,4617.314369,191.426914,240.990431,0.025859,0,0,0,0,0
77,West Town,2035.786466,17377.696952,24105.544654,7756.589542,328.084254,0,0,0,303.082042,0,7.124673,0
78,"Whitney M. Young, Jr.",2537.689200,16282.989005,19848.688504,6548.158559,0,1.363451,0.345234,2.00257,633.9218,8.480053,17.346121,0
79,Woodson Regional Library,601.143131,11216.771447,19394.908090,11632.631256,755.207743,0.997077,1.446596,0,38.540417,0,0,0


In [165]:
allRadiiPops = allRadiiPops.fillna(0)

In [166]:
allRadiiPops.head(4)

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library
0,Albany Park,743.233490,11461.490870,16937.056665,1629.263526,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0
1,Altgeld,171.224109,1333.984489,3361.805928,13984.505678,441.419500,19.440495,0.213771,0.0,0.0,0.0,0.0,0
2,Archer Heights,1768.226150,10075.564142,11282.105777,3485.545272,564.116169,0.000000,0.000000,0.0,0.0,0.0,0.0,0
3,Austin,896.808562,14412.841196,13947.921464,7484.997177,15399.165208,0.000000,0.000000,0.0,0.0,0.0,0.0,0


In [167]:
allRadiiPops = allRadiiPops.set_index('Branch')

In [168]:
allRadiiPops['Total Pop Around Library'] = allRadiiPops.sum(axis=1)

In [169]:
allRadiiPops

,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library
Branch,,,,,,,,,,,,
Albany Park,743.233490,11461.490870,16937.056665,1629.263526,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,30771.044551
Altgeld,171.224109,1333.984489,3361.805928,13984.505678,441.419500,19.440495,0.213771,0.00000,0.000000,0.000000,0.000000,19312.593971
Archer Heights,1768.226150,10075.564142,11282.105777,3485.545272,564.116169,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,27175.557511
Austin,896.808562,14412.841196,13947.921464,7484.997177,15399.165208,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,52141.733607
Austin-Irving,1419.246097,13355.359822,11761.164473,198.400952,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,26734.171344
...,...,...,...,...,...,...,...,...,...,...,...,...
West Pullman,768.427459,8982.612539,32331.153537,4617.314369,191.426914,240.990431,0.025859,0.00000,0.000000,0.000000,0.000000,47131.951108
West Town,2035.786466,17377.696952,24105.544654,7756.589542,328.084254,0.000000,0.000000,0.00000,303.082042,0.000000,7.124673,51913.908583
"Whitney M. Young, Jr.",2537.689200,16282.989005,19848.688504,6548.158559,0.000000,1.363451,0.345234,2.00257,633.921800,8.480053,17.346121,45880.984498


In [170]:
allRadiiPops.reset_index(inplace = True)

In [171]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library
0,Albany Park,743.233490,11461.490870,16937.056665,1629.263526,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,30771.044551
1,Altgeld,171.224109,1333.984489,3361.805928,13984.505678,441.419500,19.440495,0.213771,0.00000,0.000000,0.000000,0.000000,19312.593971
2,Archer Heights,1768.226150,10075.564142,11282.105777,3485.545272,564.116169,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,27175.557511
3,Austin,896.808562,14412.841196,13947.921464,7484.997177,15399.165208,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,52141.733607
4,Austin-Irving,1419.246097,13355.359822,11761.164473,198.400952,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,26734.171344
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,768.427459,8982.612539,32331.153537,4617.314369,191.426914,240.990431,0.025859,0.00000,0.000000,0.000000,0.000000,47131.951108
77,West Town,2035.786466,17377.696952,24105.544654,7756.589542,328.084254,0.000000,0.000000,0.00000,303.082042,0.000000,7.124673,51913.908583
78,"Whitney M. Young, Jr.",2537.689200,16282.989005,19848.688504,6548.158559,0.000000,1.363451,0.345234,2.00257,633.921800,8.480053,17.346121,45880.984498
79,Woodson Regional Library,601.143131,11216.771447,19394.908090,11632.631256,755.207743,0.997077,1.446596,0.00000,38.540417,0.000000,0.000000,43641.645757


In [172]:
allRadiiPops['Prop 0 to 1 Min'] = allRadiiPops['Prop 0 to 1 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 1 to 3 Min'] = allRadiiPops['Prop 1 to 3 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 3 to 5 Min'] = allRadiiPops['Prop 3 to 5 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 5 to 7 Min'] = allRadiiPops['Prop 5 to 7 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 7 to 9 Min'] = allRadiiPops['Prop 7 to 9 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 9 to 11 Min'] = allRadiiPops['Prop 9 to 11 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 11 to 13 Min'] = allRadiiPops['Prop 11 to 13 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 13 to 15 Min'] = allRadiiPops['Prop 13 to 15 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 15 to 17 Min'] = allRadiiPops['Prop 15 to 17 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 17 to 19 Min'] = allRadiiPops['Prop 17 to 19 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 19 to 21 Min'] = allRadiiPops['Prop 19 to 21 Min']/allRadiiPops['Total Pop Around Library']
#allRadiiPops['Prop 21 to 23 Min'] = allRadiiPops['Prop 21 to 23 Min']/allRadiiPops['Total Pop Around Library']
#allRadiiPops['Prop 23 to 25 Min'] = allRadiiPops['Prop 23 to 25 Min']/allRadiiPops['Total Pop Around Library']
#allRadiiPops['Prop 25 to 27 Min'] = allRadiiPops['Prop 25 to 27 Min']/allRadiiPops['Total Pop Around Library']
#allRadiiPops['Prop 27 to 30 Min'] = allRadiiPops['Prop 27 to 30 Min']/allRadiiPops['Total Pop Around Library']

In [173]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library
0,Albany Park,0.024154,0.372476,0.550422,0.052948,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,30771.044551
1,Altgeld,0.008866,0.069073,0.174073,0.724113,0.022857,0.001007,1.106900e-05,0.000000,0.000000,0.000000,0.000000,19312.593971
2,Archer Heights,0.065067,0.370758,0.415156,0.128260,0.020758,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27175.557511
3,Austin,0.017199,0.276417,0.267500,0.143551,0.295333,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,52141.733607
4,Austin-Irving,0.053087,0.499561,0.439930,0.007421,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,26734.171344
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.016304,0.190584,0.685971,0.097966,0.004062,0.005113,5.486582e-07,0.000000,0.000000,0.000000,0.000000,47131.951108
77,West Town,0.039215,0.334741,0.464337,0.149413,0.006320,0.000000,0.000000e+00,0.000000,0.005838,0.000000,0.000137,51913.908583
78,"Whitney M. Young, Jr.",0.055310,0.354896,0.432613,0.142721,0.000000,0.000030,7.524562e-06,0.000044,0.013817,0.000185,0.000378,45880.984498
79,Woodson Regional Library,0.013775,0.257020,0.444413,0.266549,0.017305,0.000023,3.314715e-05,0.000000,0.000883,0.000000,0.000000,43641.645757


In [174]:
allRadiiPops['Weighted Average Travel Time'] = 0

In [175]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.024154,0.372476,0.550422,0.052948,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,30771.044551,0
1,Altgeld,0.008866,0.069073,0.174073,0.724113,0.022857,0.001007,1.106900e-05,0.000000,0.000000,0.000000,0.000000,19312.593971,0
2,Archer Heights,0.065067,0.370758,0.415156,0.128260,0.020758,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27175.557511,0
3,Austin,0.017199,0.276417,0.267500,0.143551,0.295333,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,52141.733607,0
4,Austin-Irving,0.053087,0.499561,0.439930,0.007421,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,26734.171344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.016304,0.190584,0.685971,0.097966,0.004062,0.005113,5.486582e-07,0.000000,0.000000,0.000000,0.000000,47131.951108,0
77,West Town,0.039215,0.334741,0.464337,0.149413,0.006320,0.000000,0.000000e+00,0.000000,0.005838,0.000000,0.000137,51913.908583,0
78,"Whitney M. Young, Jr.",0.055310,0.354896,0.432613,0.142721,0.000000,0.000030,7.524562e-06,0.000044,0.013817,0.000185,0.000378,45880.984498,0
79,Woodson Regional Library,0.013775,0.257020,0.444413,0.266549,0.017305,0.000023,3.314715e-05,0.000000,0.000883,0.000000,0.000000,43641.645757,0


In [176]:
for i in range(len(allRadiiPops)):
    allRadiiPops.loc[i,'Weighted Average Travel Time'] = (((allRadiiPops.loc[i,'Prop 0 to 1 Min'])*1)+
                                                        ((allRadiiPops.loc[i,'Prop 1 to 3 Min'])*2) +
                                                        ((allRadiiPops.loc[i,'Prop 3 to 5 Min'])*4) +
                                                        ((allRadiiPops.loc[i,'Prop 5 to 7 Min'])*6) +
                                                        ((allRadiiPops.loc[i,'Prop 7 to 9 Min'])*8) +
                                                        ((allRadiiPops.loc[i,'Prop 9 to 11 Min'])*10) +
                                                        ((allRadiiPops.loc[i,'Prop 11 to 13 Min'])*12) +
                                                        ((allRadiiPops.loc[i,'Prop 13 to 15 Min'])*14) +
                                                        ((allRadiiPops.loc[i,'Prop 15 to 17 Min'])*16) +
                                                        ((allRadiiPops.loc[i,'Prop 17 to 19 Min'])*18) +
                                                        ((allRadiiPops.loc[i,'Prop 19 to 21 Min'])*20))

In [177]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.024154,0.372476,0.550422,0.052948,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,30771.044551,3.288482
1,Altgeld,0.008866,0.069073,0.174073,0.724113,0.022857,0.001007,1.106900e-05,0.000000,0.000000,0.000000,0.000000,19312.593971,5.381037
2,Archer Heights,0.065067,0.370758,0.415156,0.128260,0.020758,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27175.557511,3.402836
3,Austin,0.017199,0.276417,0.267500,0.143551,0.295333,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,52141.733607,4.864002
4,Austin-Irving,0.053087,0.499561,0.439930,0.007421,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,26734.171344,2.856458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.016304,0.190584,0.685971,0.097966,0.004062,0.005113,5.486582e-07,0.000000,0.000000,0.000000,0.000000,47131.951108,3.812780
77,West Town,0.039215,0.334741,0.464337,0.149413,0.006320,0.000000,0.000000e+00,0.000000,0.005838,0.000000,0.000137,51913.908583,3.609233
78,"Whitney M. Young, Jr.",0.055310,0.354896,0.432613,0.142721,0.000000,0.000030,7.524562e-06,0.000044,0.013817,0.000185,0.000378,45880.984498,3.584829
79,Woodson Regional Library,0.013775,0.257020,0.444413,0.266549,0.017305,0.000023,3.314715e-05,0.000000,0.000883,0.000000,0.000000,43641.645757,4.057953


In [178]:
allRadiiPops.sort_values(by=['Weighted Average Travel Time'], ascending=True)

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library,Weighted Average Travel Time
14,Canaryville,0.076115,0.755261,0.166417,0.000104,0.000000,0.000345,0.000229,0.000214,0.001182,0.000045,0.000089,4633.553985,2.283612
42,Little Italy,0.156427,0.650606,0.155526,0.035208,0.000000,0.000218,0.000030,0.000006,0.001515,0.000464,0.000000,12053.802513,2.326216
10,Brainerd,0.051128,0.600583,0.345393,0.002895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15089.842827,2.651240
15,Chicago Bee,0.106478,0.653305,0.155212,0.037768,0.021190,0.008131,0.006640,0.006381,0.003262,0.001423,0.000211,8404.191307,2.762408
6,Back of the Yards,0.042379,0.554934,0.402687,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18328.397996,2.762995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,Jeffery Manor,0.026812,0.150014,0.188367,0.318805,0.208040,0.071617,0.033617,0.001760,0.000711,0.000104,0.000152,28162.149012,5.817966
23,Dunning,0.017769,0.176744,0.341468,0.137466,0.131799,0.012205,0.042077,0.011569,0.127628,0.001275,0.000000,69964.274330,6.470248
55,Oriole Park,0.001678,0.059846,0.257953,0.279260,0.213737,0.071240,0.094776,0.012618,0.007090,0.001802,0.000000,43147.115404,6.710882
58,Roden,0.012833,0.121014,0.334076,0.250409,0.065072,0.018336,0.059984,0.027087,0.068367,0.042821,0.000000,85630.038867,6.761241


In [179]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.024154,0.372476,0.550422,0.052948,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,30771.044551,3.288482
1,Altgeld,0.008866,0.069073,0.174073,0.724113,0.022857,0.001007,1.106900e-05,0.000000,0.000000,0.000000,0.000000,19312.593971,5.381037
2,Archer Heights,0.065067,0.370758,0.415156,0.128260,0.020758,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27175.557511,3.402836
3,Austin,0.017199,0.276417,0.267500,0.143551,0.295333,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,52141.733607,4.864002
4,Austin-Irving,0.053087,0.499561,0.439930,0.007421,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,26734.171344,2.856458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.016304,0.190584,0.685971,0.097966,0.004062,0.005113,5.486582e-07,0.000000,0.000000,0.000000,0.000000,47131.951108,3.812780
77,West Town,0.039215,0.334741,0.464337,0.149413,0.006320,0.000000,0.000000e+00,0.000000,0.005838,0.000000,0.000137,51913.908583,3.609233
78,"Whitney M. Young, Jr.",0.055310,0.354896,0.432613,0.142721,0.000000,0.000030,7.524562e-06,0.000044,0.013817,0.000185,0.000378,45880.984498,3.584829
79,Woodson Regional Library,0.013775,0.257020,0.444413,0.266549,0.017305,0.000023,3.314715e-05,0.000000,0.000883,0.000000,0.000000,43641.645757,4.057953


In [180]:
averageDrivingTimeNeighborhoods = pd.DataFrame(columns=['Branch', 'Average Driving Time (Minutes)'])


In [181]:
averageDrivingTimeNeighborhoods['Branch'] = allRadiiPops['Branch']
averageDrivingTimeNeighborhoods['Average Driving Time (Minutes)'] = allRadiiPops['Weighted Average Travel Time']

In [182]:
averageDrivingTimeNeighborhoods

,Branch,Average Driving Time (Minutes)
0,Albany Park,3.288482
1,Altgeld,5.381037
2,Archer Heights,3.402836
3,Austin,4.864002
4,Austin-Irving,2.856458
...,...,...
76,West Pullman,3.812780
77,West Town,3.609233
78,"Whitney M. Young, Jr.",3.584829
79,Woodson Regional Library,4.057953


In [183]:
averageDrivingTimeNeighborhoods.sort_values(by=['Average Driving Time (Minutes)'], ascending=True)

,Branch,Average Driving Time (Minutes)
14,Canaryville,2.283612
42,Little Italy,2.326216
10,Brainerd,2.651240
15,Chicago Bee,2.762408
6,Back of the Yards,2.762995
...,...,...
36,Jeffery Manor,5.817966
23,Dunning,6.470248
55,Oriole Park,6.710882
58,Roden,6.761241


In [184]:
averageDrivingTimeNeighborhoods.to_csv('../library neighborhoods/averageDrivingTimeNeighborhoods.csv')